In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split
from sklearn import *
from scipy import *
from sklearn.decomposition import TruncatedSVD
import nltk

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\lda.py:6: Depreca

In [2]:
# Load the list of documents
train_var = pd.read_csv("training_variants")
test_var = pd.read_csv("test_variants")

# Reading train
train_text = pd.read_csv(
    'training_text', sep="\|\|", engine='python', header=None, skiprows=1, 
    names=["ID","Text"])
#read test
test_text = pd.read_csv(
    'test_text', sep="\|\|", engine='python', header=None, skiprows=1, 
    names=["ID","Text"])

#merge text and variants
train = pd.merge(train_var,train_text)
test = pd.merge(test_var,test_text)

In [39]:
#gen_var_lst = sorted(list(train.Gene.unique()) + list(train.Variation.unique()))
#print(len(gen_var_lst))
#gen_var_lst = [x for x in gen_var_lst if len(x.split(' '))==1]
#print(len(gen_var_lst))
#i_ = 0

3260
3091


In [3]:
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all['Gene_Share'] = df_all.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)
df_all['Variation_Share'] = df_all.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)

for c in df_all.columns:
    if df_all[c].dtype == 'object':
        if c in ['Gene','Variation']:
            lbl = preprocessing.LabelEncoder()
            df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
        elif c != 'Text':
            lbl = preprocessing.LabelEncoder()
            df_all[c] = lbl.fit_transform(df_all[c].values)
        if c=='Text': 
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 

train = df_all.iloc[:len(train)]
test = df_all.iloc[len(train):]

In [4]:
#data treatment
X_train = train['Text']
X_test = test['Text']
y_train = train['Class']-1
all_data = pd.concat((X_train,X_test))

In [26]:
#data split for cv
#X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size=0.33, random_state=201612)

In [5]:
#for CV
#simple method
#all_data = pd.concat((X_train,X_test))
#vect2 = CountVectorizer(min_df = 5).fit(all_data)
vect2 = CountVectorizer(min_df = 5, ngram_range=(1,8), max_features=500).fit(all_data)
#vect2 = TfidfVectorizer(min_df= 5,ngram_range=(1,3)).fit(all_data)

In [13]:
#X_train_vect2_cv = vect2.transform(X_train_cv).toarray()
#X_test_vect2_cv = vect2.transform(X_test_cv).toarray()
#X_test_vect_lgbm = vect2.transform(X_test).toarray()

In [12]:
#vcount vectorize for cv loop using X_train
vect2 = CountVectorizer(min_df = 5, ngram_range=(1,8), max_features=500).fit(all_data)
X_train_vect2_cvloop = vect2.transform(X_train)#.toarray()
X_test_vect_lgbm = vect2.transform(X_test)#.toarray()

In [9]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [10]:
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Gene_lbl_enc'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Gene_len'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Gene_words'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Gene_Share'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Variation_lbl_enc'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Variation_len'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Variation_words'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Variation_Share'])
X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Text_len'])
#X_train_vect2_cvloop = add_feature(X_train_vect2_cvloop, train['Text_words'])
#X_test_vect_lgbm
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Gene_lbl_enc'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Gene_len'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Gene_words'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Gene_Share'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Variation_lbl_enc'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Variation_len'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Variation_words'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Variation_Share'])
X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Text_len'])
#X_test_vect_lgbm = add_feature(X_test_vect_lgbm, test['Text_words'])
# convert to array
X_train_vect2_cvloop = X_train_vect2_cvloop.toarray()
X_test_vect_lgbm = X_test_vect_lgbm.toarray()

In [93]:
#svdT = TruncatedSVD(n_components=30)
#svdTFit = svdT.fit(X_train_vect2_cvloop)
#svdtest = svdT.transform(X_test_vect_lgbm)
#svdtrain = svdT.transform(X_train_vect2_cvloop)

In [11]:
# LightGBM parameters
denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 9,
        'learning_rate': 0.025,
        'max_depth': 15,
        'num_leaves': 95,
        'min_data_in_leaf': 80,
        'lambda_l1': 1.0,
        'num_iteration': 2000,
        'feature_fraction': 0.9, 
        'bagging_fraction': 0.9, 
        'bagging_freq': 5
        #'verbose': 0
    }
    X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train_vect2_cvloop, y_train, test_size=0.20, random_state=i)
    #X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(svdtrain, y_train, test_size=0.18, random_state=i)
    #X_train_vect2_cv = vect2.transform(X_train_cv).toarray()
    #X_test_vect2_cv = vect2.transform(X_test_cv).toarray()

    #lgb_train = lgb.Dataset(X_train_vect2_cv, y_train_cv)
    #lgb_eval = lgb.Dataset(X_test_vect2_cv, y_test_cv, reference=lgb_train)
    
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv)
    lgb_eval = lgb.Dataset(X_test_cv, y_test_cv, reference=lgb_train)

# train
    gbm = lgb.train(params,
            lgb_train,
            #num_boost_round=rnds,
            valid_sets=lgb_eval,
            verbose_eval=50,
            early_stopping_rounds=20)
    #score1 = metrics.log_loss(y_test_cv, gbm.predict(X_test_cv, num_iteration=gbm.best_iteration), labels = list(range(9)))
    #if score < 0.9:
    if denom != 0:
        pred = gbm.predict(X_test_vect_lgbm, num_iteration=gbm.best_iteration+50)
        #pred = gbm.predict(svdtest, num_iteration=gbm.best_iteration+50)
        
        preds += pred
    else:
        pred = gbm.predict(X_test_vect_lgbm, num_iteration=gbm.best_iteration+50)
        #pred = gbm.predict(svdtest, num_iteration=gbm.best_iteration+50)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission = submission.reset_index()
    submission.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
    #submission['ID'] = pid
    submission.to_csv('submission_xgb_fold_'  + str(i) + '.csv', index=False)
    
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission = submission.reset_index()
submission.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
#submission['ID'] = pid
submission.to_csv("sub_LGBM.csv", index=False)
#y_pred = gbm.predict(X_test_vect_lgbm, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds.
[50]	valid_0's multi_logloss: 1.43433
[100]	valid_0's multi_logloss: 1.19158
[150]	valid_0's multi_logloss: 1.08006
[200]	valid_0's multi_logloss: 1.02213
[250]	valid_0's multi_logloss: 0.992197
[300]	valid_0's multi_logloss: 0.979258
[350]	valid_0's multi_logloss: 0.973452
Early stopping, best iteration is:
[375]	valid_0's multi_logloss: 0.971955
Training until validation scores don't improve for 20 rounds.
[50]	valid_0's multi_logloss: 1.41422
[100]	valid_0's multi_logloss: 1.17813
[150]	valid_0's multi_logloss: 1.07522
[200]	valid_0's multi_logloss: 1.02272
[250]	valid_0's multi_logloss: 1.00569
[300]	valid_0's multi_logloss: 0.997833
[350]	valid_0's multi_logloss: 0.993501
Early stopping, best iteration is:
[330]	valid_0's multi_logloss: 0.991972
Training until validation scores don't improve for 20 rounds.
[50]	valid_0's multi_logloss: 1.40019
[100]	valid_0's multi_logloss: 1.14322
[150]	valid_0's multi_logloss: 1.03111

In [12]:
df = pd.DataFrame(data=y_pred[0:,0:], columns=["class1","class2","class3","class4","class5","class6","class7","class8","class9"])
df = df.reset_index()
df.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
df.to_csv("sub_5_LGBM.csv", index=False)

In [32]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
vect_tfidf = TfidfVectorizer().fit(all_data)

In [33]:
X_train_vect_tfidf_cv = vect_tfidf.transform(X_train_cv).toarray()
X_test_vect_tfidf_cv = vect_tfidf.transform(X_test_cv).toarray()
X_test_vect_tfidf_lgbm = vect_tfidf.transform(X_test).toarray()

In [34]:
lgb_train = lgb.Dataset(X_train_vect_tfidf_cv, y_train_cv)
lgb_eval = lgb.Dataset(X_test_vect_tfidf_cv, y_test_cv, reference=lgb_train)

# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 9,
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 100,
        'verbose': 0
}

# train
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=100,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)
y_pred = gbm.predict(X_test_vect_tfidf_lgbm, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 2.02684
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 1.89971
[3]	valid_0's multi_logloss: 1.79927
[4]	valid_0's multi_logloss: 1.71384
[5]	valid_0's multi_logloss: 1.641
[6]	valid_0's multi_logloss: 1.57715
[7]	valid_0's multi_logloss: 1.52139
[8]	valid_0's multi_logloss: 1.47396
[9]	valid_0's multi_logloss: 1.4317
[10]	valid_0's multi_logloss: 1.39493
[11]	valid_0's multi_logloss: 1.36257
[12]	valid_0's multi_logloss: 1.33521
[13]	valid_0's multi_logloss: 1.30837
[14]	valid_0's multi_logloss: 1.28498
[15]	valid_0's multi_logloss: 1.26408
[16]	valid_0's multi_logloss: 1.24439
[17]	valid_0's multi_logloss: 1.22785
[18]	valid_0's multi_logloss: 1.21154
[19]	valid_0's multi_logloss: 1.19687
[20]	valid_0's multi_logloss: 1.18486
[21]	valid_0's multi_logloss: 1.17331
[22]	valid_0's multi_logloss: 1.16393
[23]	valid_0's multi_logloss: 1.15552
[24]	valid_0's multi_logloss: 1.14686
[25]	valid_0's multi_logloss: 1.14038


In [36]:
y_pred = gbm.predict(X_test_vect_tfidf_lgbm, num_iteration=gbm.best_iteration)

In [37]:
df = pd.DataFrame(data=y_pred[0:,0:], columns=["class1","class2","class3","class4","class5","class6","class7","class8","class9"])
df = df.reset_index()
df.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
df.to_csv("sub_6_LGBM_TFIDF.csv", index=False)

In [32]:
#ngram vectorize for cv loop using X_train
vect_ngram = CountVectorizer(min_df = 5, analyzer= 'char_wb',ngram_range=(1,3)).fit(X_train)
X_train_vect2_cvloop_ngram = vect_ngram.transform(X_train).toarray()
#ngram for prediction using X_test
X_test_vect_ngram_lgbm = vect_ngram.transform(X_test).toarray()

In [33]:
# ngram LightGBM parameters
denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 9,
        'learning_rate': 0.025,
        'max_depth': 10,
        'num_leaves': 95,
        'min_data_in_leaf': 60,
        'lambda_l1': 1.0,
        'num_iteration': 400,
        'feature_fraction': 0.8, 
        'bagging_fraction': 0.8, 
        'bagging_freq': 5
        #'verbose': 0
    }
    X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train_vect2_cvloop_ngram, y_train, test_size=0.18, random_state=i)
    #X_train_vect2_cv = vect2.transform(X_train_cv).toarray()
    #X_test_vect2_cv = vect2.transform(X_test_cv).toarray()

    #lgb_train = lgb.Dataset(X_train_vect2_cv, y_train_cv)
    #lgb_eval = lgb.Dataset(X_test_vect2_cv, y_test_cv, reference=lgb_train)
    
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv)
    lgb_eval = lgb.Dataset(X_test_cv, y_test_cv, reference=lgb_train)

# train
    gbm = lgb.train(params,
            lgb_train,
            #num_boost_round=rnds,
            valid_sets=lgb_eval,
            verbose_eval=20,
            early_stopping_rounds=20)
    #score1 = metrics.log_loss(y_test_cv, gbm.predict(X_test_cv, num_iteration=gbm.best_iteration), labels = list(range(9)))
    #if score < 0.9:
    if denom != 0:
        pred = gbm.predict(X_test_vect_ngram_lgbm, num_iteration=gbm.best_iteration+80)
        preds += pred
    else:
        pred = gbm.predict(X_test_vect_ngram_lgbm, num_iteration=gbm.best_iteration+80)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission = submission.reset_index()
    submission.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
    #submission['ID'] = pid
    submission.to_csv('submission_lgb_fold_ngram'  + str(i) + '.csv', index=False)
    
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission = submission.reset_index()
submission.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
#submission['ID'] = pid
submission.to_csv("sub_lgb_ngram.csv", index=False)
#y_pred = gbm.predict(X_test_vect_lgbm, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:100: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds.
[20]	valid_0's multi_logloss: 1.73876
[40]	valid_0's multi_logloss: 1.50357
[60]	valid_0's multi_logloss: 1.35862
[80]	valid_0's multi_logloss: 1.25938
[100]	valid_0's multi_logloss: 1.18667
[120]	valid_0's multi_logloss: 1.13301
[140]	valid_0's multi_logloss: 1.10008
[160]	valid_0's multi_logloss: 1.07369
[180]	valid_0's multi_logloss: 1.05735
[200]	valid_0's multi_logloss: 1.04176
[220]	valid_0's multi_logloss: 1.02958
[240]	valid_0's multi_logloss: 1.02021
[260]	valid_0's multi_logloss: 1.01678
[280]	valid_0's multi_logloss: 1.01456
[300]	valid_0's multi_logloss: 1.01286
Early stopping, best iteration is:
[297]	valid_0's multi_logloss: 1.01229
Training until validation scores don't improve for 20 rounds.
[20]	valid_0's multi_logloss: 1.73951
[40]	valid_0's multi_logloss: 1.49942
[60]	valid_0's multi_logloss: 1.34923
[80]	valid_0's multi_logloss: 1.24726
[100]	valid_0's multi_logloss: 1.18101
[120]	valid_0's multi_logloss:

In [34]:
#ngram
#vect_ngram = CountVectorizer(min_df = 5, analyzer= 'char_wb',ngram_range=(2,3)).fit(X_train)
#X_train_vect_ngram_cv = vect_ngram.transform(X_train_cv).toarray()
#X_test_vect_ngram_cv = vect_ngram.transform(X_test_cv).toarray()
#X_test_vect_ngram_lgbm = vect_ngram.transform(X_test).toarray()

In [83]:
#vect = CountVectorizer(min_df = 5, analyzer= 'char_wb',ngram_range=(2,5)).fit(X_train)
#X_train_vect = vect.transform(X_train)
#X_test_vect = vect.transform(X_test)

In [101]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [119]:
#add features
#とりあえずADDしないでやってみる
leng = []
for i in train['Gene']:
    ln = i
    leng.append(ln)
leng2 = []
for i in test['Gene']:
    ln = i
    leng2.append(ln)


#X_train_vect = add_feature(X_train_vect, leng)
#X_test_vect = add_feature(X_test_vect, leng2)

In [121]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=100).fit(X_train_vect,y_train)
pred = model.predict_proba(X_test_vect)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [141]:
df = pd.DataFrame(data=pred[0:,0:], columns=["class1","class2","class3","class4","class5","class6","class7","class8","class9"])  

In [143]:
df = df.reset_index()

In [145]:
df.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]

In [147]:
df.to_csv("sub_1.csv", index=False)

In [148]:
 n1 = vect.get_feature_names()

In [8]:
all_data = pd.concat((X_train,X_test))

In [11]:
#simple method
vect2 = CountVectorizer(min_df = 50).fit(all_data)
X_train_vect2 = vect2.transform(X_train)
X_test_vect2 = vect2.transform(X_test)

In [169]:
y_train.shape

(3321,)

In [172]:
from sklearn.naive_bayes import MultinomialNB

In [173]:
model2 = MultinomialNB(alpha=0.1).fit(X_train_vect2, y_train)
pred2 = model2.predict_proba(X_test_vect2)

In [175]:
df2 = pd.DataFrame(data=pred2[0:,0:], columns=["class1","class2","class3","class4","class5","class6","class7","class8","class9"])
df2 = df2.reset_index()
df2.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
df2.to_csv("sub_2_MNB.csv", index=False)

In [180]:
from sklearn.svm import SVC
model_svc = SVC(C=10000,probability = True).fit(X_train_vect2, y_train)
pred_svc = model_svc.predict_proba(X_test_vect2)

In [181]:
df_svc = pd.DataFrame(data=pred_svc[0:,0:], columns=["class1","class2","class3","class4","class5","class6","class7","class8","class9"])
df_svc = df_svc.reset_index()
df_svc.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
df_svc.to_csv("sub_3_SVC.csv", index=False)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
X_train_vect_GBM = vect2.transform(X_train).toarray()
X_test_vect_GBM = vect2.transform(X_test).toarray() # Update line
model_GBM = GBC(n_estimators=100,max_depth=3).fit(X_train_vect_GBM, y_train)
pred_GBM = model_GBM.predict_proba(X_test_vect_GBM)

In [187]:
df_GBM = pd.DataFrame(data=pred_GBM[0:,0:], columns=["class1","class2","class3","class4","class5","class6","class7","class8","class9"])
df_GBM = df_GBM.reset_index()
df_GBM.columns = ["ID","class1","class2","class3","class4","class5","class6","class7","class8","class9"]
df_GBM.to_csv("sub_4_GBM.csv", index=False)

In [ ]:
#from sklearn.metrics import mean_squared_error
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.model_selection import cross_val_score
import random

random.seed(1)
MSE_list_GB = []
dflist_GB = []
def GB():
    for ne in [100,200,300,400,500]:
        for md in [1,2,3,4,5]:
            model_gb = GBC(n_estimators=ne,max_depth=md)
            MSE = (-cross_val_score(model_gb, X_train_vect_GBM, y_train, cv=5, scoring = 'neg_mean_squared_error')).mean()
            MSE_list_GB.append(MSE)
            a = ne
            b = md
            c = (a,b)
            dflist_GB.append(c)
    return MSE_list_GB
GB()
df = pd.DataFrame(dflist_GB,columns=['NumEst', 'MaxDepth'])
df['MSE']=MSE_list_GB
df = df.sort_values(by ='MSE',ascending = True)
df

In [47]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
train_var = pd.read_csv("training_variants")
# Reading train
train_text = pd.read_csv(
    'training_text', sep="\|\|", engine='python', header=None, skiprows=1, 
    names=["ID","Text"])

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(train_text['Text'])

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [53]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=9, id2word=id_map, passes=25, random_state=34)

KeyboardInterrupt: 